In [ ]:
import socket
import time

from pop import Gas
from pop import Dust
from pop import TempHumi
from pop import Pir  # PIR??pop ?ºÏù¥Î∏åÎü¨Î¶¨Ïóê ?¨Ìï®?òÏñ¥ ?àÏùå

# ---------------------------
# JAVA ?úÎ≤Ñ Ï£ºÏÜå & ?¨Ìä∏
# ---------------------------
JAVA_IP = "127.0.0.1"   # ?àÎèÑ??IP
JAVA_PORT = 39187             # ???úÎ≤Ñ ?¨Ìä∏

# ---------------------------
# POP ?ºÏÑú Ï¥àÍ∏∞??n# ---------------------------
gas = Gas(6)
dust = Dust()
th = TempHumi()
pir = Pir(22)

# ---------------------------
# TCP ?∞Í≤∞
# ---------------------------
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((JAVA_IP, JAVA_PORT))
print("[PY] JAVA ?úÎ≤Ñ ?∞Í≤∞ ?ÑÎ£å")

# ---------------------------
# ?ºÏÑúÍ∞?Ï∏°Ï†ï + ?ÑÏÜ° Î£®ÌîÑ
# ---------------------------
while True:
    # ---- Gas ----
    gas_val = gas.readAverage()
    methan = gas.calcMethan(gas_val)

    # ---- Dust ----
    dust.read()
    pm1  = dust.pm_1p0_grimm
    pm25 = dust.pm_2p5_grimm
    pm10 = dust.pm_10_grimm

    # ---- Temp & Humi ----
    temp = th.getTemperature()
    humi = th.getHumidity()

    # ---- PIR ----
    pir_val = pir.read()
    # 1 = movement, 0 = no movement

    # ---- JAVAÎ°??ÑÏÜ°???®ÌÇ∑ ?ùÏÑ± ----
    message = (
        f"SENSOR "
        f"GAS={gas_val} "
        f"METHAN={methan} "
        f"TEMP={temp:.2f} "
        f"HUMI={humi:.2f} "
        f"PM1={pm1} "
        f"PM25={pm25} "
        f"PM10={pm10} "
        f"PIR={pir_val}"
    )

    try:
        sock.sendall((message + "\n").encode())
    except:
        print("[PY] ?úÎ≤Ñ ?∞Í≤∞ ?äÍ?. ?¨Ïó∞Í≤??úÎèÑ Ï§?..")
        time.sleep()
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((JAVA_IP, JAVA_PORT))
        continue

    time.sleep(1)   # 1Ï¥àÎßà???ÑÏÜ°
